# Data retrieving of the bryozoan coverage on the kelp lamina
**Author:** Jelte Doeksen\
**Co-Author:** Elisabeth Snijder 

![](https://i0.wp.com/dianaurban.com/wp-content/uploads/2017/10/cat-typing.gif?resize=400%2C400&ssl=1)

Feature implementation wishlist:
- Convert svg selections to pytorch rectangles for training models
- Use Segment Anything Model to find more accurate pixel sizes


## Install dependencies

First make sure the dependencies are installed by running the following two commands in the terminal:

<!-- `pip install git+https://github.com/facebookresearch/segment-anything.git` -->

`pip install opencv-python matplotlib pandas openpyxl ipywidgets`

<!-- Then download the [default SAM model](https://dl.fbaipublicfiles.com/segment_anything/sam_vit_h_4b8939.pth) and place it in the same folder as this notebook. -->



## Starting notebook

Now, it's time to import the packages into your current Python kernel.

In [ ]:
import numpy as np
import pandas as pd
import os
import re
import math
import matplotlib.pyplot as plt
import cv2
from datetime import date
from xml.dom import minidom
from scipy import integrate
from ipywidgets import IntProgress
from IPython.display import display

Setting the backround to transparant for nicer plot and widget outputs in the notebook

In [ ]:
%%html
<style>
.cell-output-ipywidget-background {
   background-color: transparent !important;
}
.jp-OutputArea-output {
   background-color: transparent;
}  
</style>

## Preparing the inputs

The next step is to prepare which pictures this script will run on, and to configure some specific settings from the manual selection work.

The **`footage_dir`** is the path to the bryozoan scans in .tif format. This needs to be filled in.
The `footage_list` is then generated by listing all the files for which also a scaled version ending with " (kelp)" and an SVG file are available. The list does not contain a file extension.
From the `footage_list`, a `specimen_list` is extracted. This list contains the specimen id's, which consist of the first 4 characters of the file names, and represents the individual kelp leaves that were taken from the farm, before they were cut into scannable sections.

The original dpi setting is entered in **`footage_dpi`**, to translate between pixels and millimeters.

Next up is a list of dictionaries called **`feature_types`**, which holds the key information to distinguish between different types of biofouling features. Also a mapping is defined called **`svg_mapping`**, which will be used to translate the properties of the different SVG shapes into generic column values. Be aware that for example `size.x` may mean something different depending on the selected biofouling type.

List **`sample_dates`** holds the dates on which specimens were collected from the farm.

<!-- Finally, **`deployment_dates`** contains the dates on which the lines were deployed where the specimens originate from. -->

In [ ]:
# Fill in footage_dir with the folder that contains the scans. Remember to use double backslashes on Windows
footage_dir = os.environ["FOOTAGE_DIR"]
dir_separator = os.environ['DIR_SEPARATOR']
svg_list = [ file.name.removesuffix('.svg') for file in os.scandir(footage_dir) if file.is_file() and file.name.endswith('.svg')]
raw_footage_list = [ file.name.removesuffix('.tif') for file in os.scandir(footage_dir) if file.is_file() and file.path.endswith('.tif') and not file.path.endswith(').tif')]
footage_list = set(svg_list) & set(raw_footage_list)
specimen_list = [] # set([scan_id[:4] for scan_id in footage_list]) # Easy option: just take first 4 characters
for scan_id in footage_list: # Advanced option: take first two numbers, in case a new sampling campaign takes 10+ specimens per date and/or samples on 10+ dates.
    scan_id_numbers = re.findall(r'\d+', scan_id)
    specimen_list.append(f"{scan_id_numbers[0]}SP{scan_id_numbers[1]}")
specimen_list = set(specimen_list)

# Original footage dpi setting
footage_dpi = int(os.environ["FOOTAGE_DPI"])

# Define feature_types for recognizing area's from the SVG files
feature_types = [
    {
        "type": "undetermined",
        "shape": "circle",
        "color": "#ff0c0c",
    },
    {
        "type": "recruitment",
        "shape": "ellipse",
        "color": "#1200d8",
    },
    {
        "type": "colony_top",
        "shape": "ellipse",
        "color": "#000000",
    },
    {
        "type": "colony_bottom",
        "shape": "ellipse",
        "color": "#03ce00",
    },
    #{
        # "type": "negative",
        # "shape": "rectangle",
        # "color": "#000000",
    # },
]

# Map specific properties of SVG elements to generic column names, including formula's for calculating surface area
svg_mapping = {
    'circle': {
        'location.x': 'cx',
        'location.y': 'cy',
        'size.x': 'r',
        'size.y': 'r',
        'area': lambda sx, sy: math.pi * pow(sx, 2),
    },
    'ellipse': {
        'location.x': 'cx',
        'location.y': 'cy',
        'size.x': 'rx',
        'size.y': 'ry',
        'area': lambda sx, sy: math.pi * sx * sy,
    },
    # 'rectangle': {
    #     'location.x': 'x',
    #     'location.y': 'y',
    #     'size.x': 'width',
    #     'size.y': 'height',
    #     'area': lambda sx, sy: sx * sy,
    # },
}

# Define dates on which specimens have been collected
sample_dates = [
    date(2023,4,25),
    date(2023,5,10),
    date(2023,5,15),
    date(2023,5,23),
    date(2023,6,7),
    date(2023,6,15),
    date(2023,6,29),
]

## Preparing the outputs

Now the output dataframes are prepared. The iterators in the next sections will fill these dataframes, after which they can be exported to an excelsheet.

The dataframes are:

`features`, listing all the selected features on all scans for which there is also an SVG and resized image available:
- scan_id
- specimen_id
- sample_date
- type
- path_name
- shape
- color
- location.x
- location.y
- size.x
- size.y
- rotation
- area.sq_pixels
- area.sq_millimeter
- area.sq_meter

`scan_aggregates`, listing the statistics for each scan, such as total kelp area on that scan, percentage coverage and counts per feature type:
- scan_id
- specimen_id
- single_scan
- sample_date
- counts.total
- counts.undetermined
- counts.recruitments
- counts.colonies
- colony_size.mean
- colony_size.min
- colony_size.max
- colony_size.std
- scan_area.sq_pixels
- scan_area.sq_millimeter
- scan_area.sq_meter
- kelp_area.percentage_of_scan_area
- kelp_area.sq_pixels
- kelp_area.sq_millimeter
- kelp_area.sq_meter
- kelp_area.sq_meter_coverage
- kelp_area.percentage_coverage
- counts_per_sq_meter.total
- counts_per_sq_meter.undetermined
- counts_per_sq_meter.recruitments
- counts_per_sq_meter.colonies

`specimen_aggregates`, a summary of scan aggregates aggregating all scans of a specimen:
- specimen_id
- sample_date
- counts.total
- counts.undetermined
- counts.recruitments
- counts.colonies
- colony_size.mean
- colony_size.min
- colony_size.max
- colony_size.std
- kelp_area.sq_meter
- kelp_area.sq_meter_coverage
- kelp_area.percentage_coverage
- counts_per_sq_meter.total
- counts_per_sq_meter.undetermined
- counts_per_sq_meter.recruitments
- counts_per_sq_meter.colonies

`event_aggregates`, aggregating one step higher, combining all information per date on which samples were collected from the farm, and adding statistics from the continuous measurements that were recorded in between these dates:
- sample_date
- number_of_specimens
- number_of_scans
- counts.total
- counts.undetermined
- counts.recruitments
- counts.colonies
- colony_size.mean
- colony_size.min
- colony_size.max
- colony_size.std
- kelp_area.sq_pixels_scaled
- kelp_area.sq_pixels
- kelp_area.sq_millimeter
- kelp_area.sq_meter
- kelp_area.sq_meter_coverage
- kelp_area.percentage_coverage
- counts_per_sq_meter.total
- counts_per_sq_meter.undetermined
- counts_per_sq_meter.recruitments
- counts_per_sq_meter.colonies

<!-- `raw_timeseries`, containing timeseries data from CTD and other sensors:
- datetime
- temperature
- turbidity
- chlorophyl -->

In [ ]:
# Create empty dataframes
features = pd.DataFrame()
scan_aggregates = pd.DataFrame()
specimen_aggregates = pd.DataFrame()
event_aggregates = pd.DataFrame()

## Iterating over selected footage to fill features, scan, specimen and event aggregates

Now we start filling the dataframes. It starts with the simple ones by retrieving the contents of the SVG-files and aggregating that. In addition, OpenCV is used to transform the images into grayscale and a threshold is set to select what is kelp (the measurement tape is compensated for by deducting a fixed number of pixels).

In [ ]:
# Initialize progress bar
progress = IntProgress(min=0, max=len(footage_list))
display(progress)
# First, iterate over all the SVG files and filter and add the contents to the features dataframe
for scan_id in footage_list:
    # Determine the sample date and specimen id of this scan
    scan_id_numbers = re.findall(r'\d+', scan_id)
    sample_date = sample_dates[int(scan_id_numbers[0])-1]
    specimen_id = f"{scan_id_numbers[0]}SP{scan_id_numbers[1]}"
    
    # Read SVG file
    svg_doc = minidom.parse(f"{footage_dir}{dir_separator}{scan_id}.svg")
    for feature_type in feature_types:
        # For each feature, get the elements of corresponding shape and color
        feature_elements = [element for element in svg_doc.getElementsByTagName(feature_type['shape']) if f"stroke:{feature_type['color']};" in element.getAttribute('style')]
        for feature_element in feature_elements:
            # Extract some values for which the SVG attribute depends on the shape
            location_x = float(feature_element.getAttribute(svg_mapping[feature_type['shape']]['location.x']))
            location_y = float(feature_element.getAttribute(svg_mapping[feature_type['shape']]['location.y']))
            size_x = float(feature_element.getAttribute(svg_mapping[feature_type['shape']]['size.x']))
            size_y = float(feature_element.getAttribute(svg_mapping[feature_type['shape']]['size.y']))
            rotation = re.search('rotate\((.*)\)', feature_element.getAttribute('transform'))
            area = svg_mapping[feature_type['shape']]['area'](size_x, size_y)
            # For each element, add a samples entry
            feaatures_entry = pd.DataFrame({
                'scan_id': scan_id,
                'specimen_id': specimen_id,
                'sample_date': sample_date,
                'type': feature_type['type'],
                'path_name': feature_element.getAttribute('id'),
                'shape': feature_type['shape'],
                'color': feature_type['color'],
                'location.x': location_x,
                'location.y': location_y,
                'size.x': size_x,
                'size.y': size_y,
                'rotation': float(rotation.group(1)) if rotation else 0.0,
                'area.sq_pixels': round(area),
            }, index=[0])
            features = pd.concat([features, feaatures_entry], ignore_index=True)
    svg_doc.unlink()
    progress.value += 1

# Now, add additional columns to the samples dataframe with derived values
features['area.sq_millimeters'] = features['area.sq_pixels'] * pow(25.4, 2) / pow(footage_dpi, 2)
features['area.sq_meters'] = features['area.sq_millimeters'] / 1000000

In [ ]:
# The second step is to loop over all scans and fill the scan_aggregates dataframe with statistics about the features on them
# Initialize progress bar
progress = IntProgress(min=0, max=len(footage_list))
display(progress)
# This loop starts the same as in the previous step, but for readability it has been split up into two cells.
for scan_id in footage_list:
    # Determine the sample date and specimen id of this scan
    scan_id_numbers = re.findall(r'\d+', scan_id)
    sample_date = sample_dates[int(scan_id_numbers[0])-1]
    specimen_id = f"{scan_id_numbers[0]}SP{scan_id_numbers[1]}"

    # Load image
    scan = cv2.imread(f"{footage_dir}{dir_separator}{scan_id}.tif")
    scan_gray = cv2.cvtColor(scan, cv2.COLOR_BGR2GRAY)
    (threshold, mask) = cv2.threshold(scan_gray, 192, 255, cv2.THRESH_BINARY)
    height, width, channels = scan.shape


    # Plot and export the image for reference
    fig = plt.figure(figsize=(5,5))
    ax = fig.gca()
    ax.imshow(mask, cmap='gray')
    fig.suptitle(f"Threshold: {threshold}", fontsize=18)
    ax.axis('off')
    fig.savefig(f"{footage_dir}{dir_separator}{scan_id} (mask).png")
    plt.close(fig)

    # Select the features on the scan and generate/fill aggregates
    features_subset = features[features['scan_id'] == scan_id]
    colony_subset = features_subset[(features_subset['type'] == 'colony_top') | (features_subset['type'] == 'colony_bottom')]
    single_scan = sum([bool(re.match(scan_id[:-1], other_scan_id)) for other_scan_id in footage_list]) == 1
    scan_aggregates_entry = pd.DataFrame({
        'scan_id': scan_id,
        'specimen_id': specimen_id,
        'single_scan': single_scan,
        'sample_date': sample_dates[int(re.findall(r'\d+', scan_id)[0])-1], # Read the number from the scan ID
        'counts.total': features_subset.shape[0],
        'counts.undetermined': features_subset[features_subset['type'] == 'undetermined'].shape[0],
        'counts.recruitments': features_subset[(features_subset['type'] == 'recruitment')].shape[0],
        'counts.colonies': colony_subset.shape[0],
        'colony_size.mean': colony_subset['area.sq_millimeters'].mean(),
        'colony_size.min': colony_subset['area.sq_millimeters'].min(),
        'colony_size.max': colony_subset['area.sq_millimeters'].max(),
        'colony_size.std': colony_subset['area.sq_millimeters'].std(),
        'scan_area.height_pixels': height,
        'scan_area.width_pixels': width,
        'kelp_area.sq_pixels': max(0, mask.astype(bool).sum()-6558000), # 6558000 is the approximate number of 'false-positive' pixels from the measurement tape for scans of the full length of the scan bed. Avoid going below zero
        'kelp_area.sq_meter_coverage': features_subset['area.sq_meters'].sum(),
    }, index=[0])
    scan_aggregates = pd.concat([scan_aggregates, scan_aggregates_entry], ignore_index=True)
    progress.value += 1


# Again, add columns with derived information afterwards
scan_aggregates['scan_area.sq_pixels'] = scan_aggregates['scan_area.height_pixels'] * scan_aggregates['scan_area.width_pixels']
scan_aggregates['scan_area.sq_millimeter'] = scan_aggregates['scan_area.sq_pixels'] * pow(25.4, 2) / pow(footage_dpi, 2)
scan_aggregates['scan_area.sq_meter'] = scan_aggregates['scan_area.sq_millimeter'] / 1000000
scan_aggregates['kelp_area.percentage_of_scan_area'] = scan_aggregates['kelp_area.sq_pixels'] / scan_aggregates['scan_area.sq_pixels'] * 100
scan_aggregates['kelp_area.sq_millimeter'] = (scan_aggregates['single_scan'] + 1) * scan_aggregates['kelp_area.sq_pixels'] * pow(25.4, 2) / pow(footage_dpi, 2) # (scan_aggregates['single_scan'] + 1) * doubles the kelp area for pieces that were analyzed from one side only
scan_aggregates['kelp_area.sq_meter'] = scan_aggregates['kelp_area.sq_millimeter'] / 1000000
scan_aggregates['kelp_area.percentage_coverage'] = scan_aggregates['kelp_area.sq_meter_coverage'] / scan_aggregates['kelp_area.sq_meter'] * 100
scan_aggregates['counts_per_sq_meter.total'] = scan_aggregates['counts.total'] / scan_aggregates['kelp_area.sq_meter']
scan_aggregates['counts_per_sq_meter.undetermined'] = scan_aggregates['counts.undetermined'] / scan_aggregates['kelp_area.sq_meter']
scan_aggregates['counts_per_sq_meter.recruitments'] = scan_aggregates['counts.recruitments'] / scan_aggregates['kelp_area.sq_meter']
scan_aggregates['counts_per_sq_meter.colonies'] = scan_aggregates['counts.colonies'] / scan_aggregates['kelp_area.sq_meter']

In [ ]:
# As a third step, the information from each scan is combined to receive the aggregates per specimen
# Initialize progress bar
progress = IntProgress(min=0, max=len(specimen_list))
display(progress)
# Loop over the specimen list
for specimen_id in specimen_list:
    # Select the subsets of scan aggregates and features from this specimen and generate/fill aggregates
    # Use as much as possible the features dataframe, instead of having to calculate weighted avarages
    features_subset = features[features['specimen_id'] == specimen_id]
    colony_subset = features_subset[(features_subset['type'] == 'colony_top') | (features_subset['type'] == 'colony_bottom')]
    scan_aggregates_subset = scan_aggregates[scan_aggregates['specimen_id'] == specimen_id]
    specimen_aggregates_entry = pd.DataFrame({
        'specimen_id': specimen_id,
        'sample_date': sample_dates[int(re.findall(r'\d+', specimen_id)[0])-1],
        'number_of_scans': scan_aggregates_subset.shape[0],
        'counts.total': features_subset.shape[0],
        'counts.undetermined': features_subset[features_subset['type'] == 'undetermined'].shape[0],
        'counts.recruitments': features_subset[(features_subset['type'] == 'recruitment')].shape[0],
        'counts.colonies': colony_subset.shape[0],
        'colony_size.mean': colony_subset['area.sq_millimeters'].mean(),
        'colony_size.min': colony_subset['area.sq_millimeters'].min(),
        'colony_size.max': colony_subset['area.sq_millimeters'].max(),
        'colony_size.std': colony_subset['area.sq_millimeters'].std(),
        'kelp_area.sq_meter': scan_aggregates_subset['kelp_area.sq_meter'].sum(),
        'kelp_area.sq_meter_coverage': features_subset['area.sq_meters'].sum(),
    }, index=[0])
    specimen_aggregates = pd.concat([specimen_aggregates, specimen_aggregates_entry])
    progress.value += 1

# Again, add columns with derived information afterwards
specimen_aggregates['kelp_area.percentage_coverage'] = 100 * specimen_aggregates['kelp_area.sq_meter_coverage'] / specimen_aggregates['kelp_area.sq_meter']
specimen_aggregates['counts_per_sq_meter.total'] = specimen_aggregates['counts.total'] / specimen_aggregates['kelp_area.sq_meter']
specimen_aggregates['counts_per_sq_meter.undetermined'] = specimen_aggregates['counts.undetermined'] / specimen_aggregates['kelp_area.sq_meter']
specimen_aggregates['counts_per_sq_meter.recruitments'] = specimen_aggregates['counts.recruitments'] / specimen_aggregates['kelp_area.sq_meter']
specimen_aggregates['counts_per_sq_meter.colonies'] = specimen_aggregates['counts.colonies'] / specimen_aggregates['kelp_area.sq_meter']

## Aggregation based on sampling events

In [ ]:
# Aggregate timeseries and per-sample aggregated data
# Initialize progress bar
progress = IntProgress(min=0, max=len(sample_dates))
display(progress)
# Loop over sample dates
for sample_date in sample_dates:
    # Select the subsets of specimen and scan aggregates and features from this sampling event and generate/fill aggregates
    # Use as much as possible the features dataframe, instead of having to calculate weighted avarages
    features_subset = features[features['sample_date'] == sample_date]
    colony_subset = features_subset[(features_subset['type'] == 'colony_top') | (features_subset['type'] == 'colony_bottom')]
    scan_aggregates_subset = scan_aggregates[scan_aggregates['sample_date'] == sample_date]
    specimen_aggregates_subset = specimen_aggregates[specimen_aggregates['sample_date'] == sample_date]
    event_aggregates_entry = pd.DataFrame({
        'sample_date': sample_date,
        'number_of_specimen': specimen_aggregates_subset.shape[0],
        'number_of_scans': scan_aggregates_subset.shape[0],
        'counts.total': features_subset.shape[0],
        'counts.undetermined': features_subset[features_subset['type'] == 'undetermined'].shape[0],
        'counts.recruitments': features_subset[(features_subset['type'] == 'recruitment')].shape[0],
        'counts.colonies': colony_subset.shape[0],
        'colony_size.mean': colony_subset['area.sq_millimeters'].mean(),
        'colony_size.min': colony_subset['area.sq_millimeters'].min(),
        'colony_size.max': colony_subset['area.sq_millimeters'].max(),
        'colony_size.std': colony_subset['area.sq_millimeters'].std(),
        'kelp_area.sq_meter': scan_aggregates_subset['kelp_area.sq_meter'].sum(),
        'kelp_area.sq_meter_coverage': features_subset['area.sq_meters'].sum(),
    }, index=[0])
    event_aggregates = pd.concat([event_aggregates, event_aggregates_entry])
    progress.value += 1

# Again, add columns with derived information afterwards
event_aggregates['kelp_area.percentage_coverage'] = 100 * event_aggregates['kelp_area.sq_meter_coverage'] / event_aggregates['kelp_area.sq_meter']
event_aggregates['counts_per_sq_meter.total'] = event_aggregates['counts.total'] / event_aggregates['kelp_area.sq_meter']
event_aggregates['counts_per_sq_meter.undetermined'] = event_aggregates['counts.undetermined'] / event_aggregates['kelp_area.sq_meter']
event_aggregates['counts_per_sq_meter.recruitments'] = event_aggregates['counts.recruitments'] / event_aggregates['kelp_area.sq_meter']
event_aggregates['counts_per_sq_meter.colonies'] = event_aggregates['counts.colonies'] / event_aggregates['kelp_area.sq_meter']

## Export obtained dataframes to Excel

The final step is to export all the tabular data to Excel for further processing. Each dataframe is exported to a separate sheet in the file, and the file is saved to the `footage_dir`.

In [ ]:
with pd.ExcelWriter(f"{footage_dir}{dir_separator}bryozoan_coverage_quantification_results.xlsx") as writer:
    features.to_excel(writer, sheet_name='features', index=False)
    scan_aggregates.to_excel(writer, sheet_name='scan_aggregates', index=False)
    specimen_aggregates.to_excel(writer, sheet_name='specimen_aggregates', index=False)
    event_aggregates.to_excel(writer, sheet_name='event_aggregates', index=False)